In [1]:
import os
import sys
import pickle
import json
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
def get_result_replay(path, epoch=10000):
    res = []
    lateral_shift = total_lateral_shift = lateral_shift_openpilot = 0
    with open(path) as f:
        for line in f:
            if ': yaw_diff:' in line:
                lateral_shift = float(line.strip().split(':')[-3].split(',')[0])
                total_lateral_shift = float(line.strip().split(':')[-2].split(',')[0])
                lateral_shift_openpilot = float(line.strip().split(':')[-1].split(',')[0])

                res.append((lateral_shift, total_lateral_shift, lateral_shift_openpilot))
            
    return pd.DataFrame(res, columns=['lateral_shift', 'total_lateral_shift', 'lateral_shift_openpilot'])


def get_result_replay2(path, epoch=10000):
    res = []
    lateral_shift = total_lateral_shift = lateral_shift_openpilot = 0
    with open(path) as f:
        for line in f:
            if '] desired steering angle:' in line:

                desire = float(line.strip().split(':')[3].split(',')[0])
                current = float(line.strip().split(':')[4])

                res.append((desire, current))
            
    return pd.DataFrame(res, columns=['desire', 'current'])

In [3]:
##### Attack
frame = 0
N = 20
M = 20
BASE_DIR = '../logs/comma_attack/'
all_res = []

for sim in tqdm([10]):
    map_res = {}
    for model_type in tqdm(['laneatt', 'scnn', 'ultrafast', 'polylanenet']):
        list_res = []
        for i in tqdm(range(1, 101)):

            df_gt = get_result_replay(f'../logs/comma_benign/replay_nan_more/scb{i}/replay_comma_more_metric.py_f0.log').tail(N).reset_index(drop=True).head(M)
            assert df_gt.shape[0] == M
            df_e2e = get_result_replay(BASE_DIR + f'logs_more_drp_wb/logs_more_{model_type}/scb{i}/run_comma_more_attack_drp_wb.py.log').tail(N).reset_index(drop=True).head(M)
            if df_e2e.shape[0] == 0:
                print(BASE_DIR + f'logs_more_drp_wb/logs_more_{model_type}/scb{i}/run_comma_more_attack_drp_wb.py.log')
                raise Exception(f'{model_type} {i}')

            metric = 0
            for frame in range(M - 1):
                df_metric  = get_result_replay(BASE_DIR + f'logs_more_drp_wb/logs_more_{model_type}/scb{i}/replay_comma_more_attack_drp_wb_metric.py_f{frame}.log').tail(N).reset_index(drop=True).head(M)
                if df_metric.shape[0] != df_gt.shape[0]:
                    raise Exception(f'{model_type} sc{i} frame={frame}')
                m = (df_metric['total_lateral_shift'] - df_gt['total_lateral_shift']).abs().iloc[frame + 1:].head(sim).max()
                metric += m / sim

            e2e = (df_e2e['total_lateral_shift'] - df_gt['total_lateral_shift']).abs().max()
            res = [frame, i, e2e, metric]
            list_res.append(res)


        df = pd.DataFrame(list_res, columns=['frame',  '#', f'{model_type}_e2e', f'{model_type}_metric'])
        r, p = pearsonr(df[f'{model_type}_e2e'].values, df[f'{model_type}_metric'].values)

        ### To obtain Accuracy and F1 score, please run tusimple-benchmark/example/xxx.ipynb, and use the following code
        # df_old = pd.read_csv(f'tusimple-benchmark/example/df_attack_acc_{model_type}.csv')
        # df_old['#'] = df_old['scn']#.map(lambda x: int(x.split('/')[-3][3:])) 
        # df_old = df_old.groupby('#', as_index=False)[['ACC', 'F1']].mean()
        # df_old = pd.merge(df, df_old, on='#', how='inner')
        # r_acc, p_acc = pearsonr(df_old[f'{model_type}_e2e'].values, df_old[f'ACC'].values)
        # r_f1, p_f1 = pearsonr(df_old[f'{model_type}_e2e'].values, df_old[f'F1'].values)
                
        all_res.append({
            'model': model_type, 
                               'sim': sim, 
                               'r': r, 'p': p,
                               'e2e': df[f'{model_type}_e2e'].mean(),
                                'metric': df[f'{model_type}_metric'].mean(),

                               'e2e_std': df[f'{model_type}_e2e'].std(),
                                'metric_std': df[f'{model_type}_metric'].std(),
            
                                # 'ACC': df_old['ACC'].mean(),
                                # 'F1': df_old['F1'].mean(),
                                # 'r_acc': r_acc, 'p_acc': p_acc,
                                # 'r_f1': r_f1, 'p_f1': p_f1,
                              })

In [4]:
df_res = pd.DataFrame(all_res)
df_res['p*'] = df_res['p'] < 0.05
df_res['p**'] = df_res['p'] < 0.1
pd.options.display.max_rows = 999
df_res

,model,sim,r,p,e2e,metric,e2e_std,metric_std,p*,p**
0,laneatt,10,0.952190,2.871914e-52,0.718546,0.803190,0.427487,0.479374,True,True
1,scnn,10,0.956611,2.751612e-54,0.480297,0.576358,0.387050,0.477149,True,True
2,ultrafast,10,0.927686,1.013717e-43,0.578595,0.616981,0.343069,0.419585,True,True
3,polylanenet,10,0.970116,4.415753e-62,0.381957,0.417396,0.289877,0.312773,True,True
